In [ ]:
import os
import sys

import folium
from google.cloud import bigquery
import polyline


sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# Get data from bigquery

In [ ]:
GCP_PROJECT_ID = os.environ.get('GCP_PROJECT_ID')
BIGQUERY_DATASET = os.environ.get('BIGQUERY_DATASET')
BIGQUERY_RAW_ACTIVITIES = os.environ.get('BIGQUERY_RAW_ACTIVITIES')

TABLE_ID = f'{GCP_PROJECT_ID}.{BIGQUERY_DATASET}.{BIGQUERY_RAW_ACTIVITIES}'
print(TABLE_ID)

In [ ]:
CREDENTIALS_PATH = '../credentials/sa-athlete-dashboard.json'

client = bigquery.Client.from_service_account_json(
    CREDENTIALS_PATH, project=GCP_PROJECT_ID
)

In [ ]:
table = client.get_table(TABLE_ID)
print(f'Tabelle gefunden: {TABLE_ID}')
print(f'Spalten: {[schema.name for schema in table.schema]}')
print(f'Zeilen: {table.num_rows}')

query = f'SELECT * FROM `{TABLE_ID}`'
print('🔹 Starte Abfrage:', query)

job = client.query(query)
df = job.result().to_dataframe()

print('Abfrage abgeschlossen, erste Zeilen:')
df

In [ ]:
polyline_string = df.map_summary_polyline[10]
polyline_string

In [ ]:
decoded_coords = polyline.decode(polyline_string)

if decoded_coords:
    center_point = decoded_coords[0]
else:
    print('Die Polyline ist leer. Es können keine Koordinaten dekodiert werden.')

m = folium.Map(location=center_point, zoom_start=13)

folium.PolyLine(locations=decoded_coords, color='blue', weight=5, opacity=0.8).add_to(m)

folium.Marker(
    location=decoded_coords[0], popup='Startpunkt', icon=folium.Icon(color='green')
).add_to(m)
folium.Marker(
    location=decoded_coords[-1], popup='Endpunkt', icon=folium.Icon(color='red')
).add_to(m)

m